<a href="https://colab.research.google.com/github/Rogerio-mack/CDA_2021_2S/blob/main/CDA_N1_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

<br>
<br>
<br>
<br>
<br>

# A1 Lab Caso: **Controle de Qualidade de Peças de uma Indústria** 
---

**pieces** é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

Seu trabalho é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos. Em seguida você empregará um modelo de regressão linear para predição de medidas das peças.

**Execute o exercício no Google Colab!**

<br>

***

<br>

Empregue os arquivos conforme o final do seu TIA:

* TIA final ímpar: Arquivo [\*] = **1**
* TIA final par: Arquivo [\*] = **2** 

> Arquivo de Exemplos: http://meusite.mackenzie.br/rogerio/TIC/pieces[*].csv

> Arquivo de Novos Casos: http://meusite.mackenzie.br/rogerio/TIC/pieces[*]_new.csv

Nos modelos não empregue nenhum parâmetro além dos informados.


<br>

***


# Q0. Aquisição dos Dados e Imports

In [ ]:
install.packages('dummies') # hot encode
library(dummies)
install.packages('class') # knn
library(class)
install.packages('rpart') # rpart (Árvore de Decisão)
library(rpart)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
f = readline('Entre com o file number (1,2,3,4 ou 5): ')

Entre com o file number (1,2,3,4 ou 5): 1


In [ ]:
arquivo = paste0('http://meusite.mackenzie.br/rogerio/TIC/pieces',f,'.csv')
df = read.csv(arquivo)
head(df)

,id,A,B,C,D,Quality,Unit
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,559,4.9,3.1,1.5,0.1,Reject,SP
2,629,4.8,3.4,1.6,0.2,Reject,SP
3,192,6.7,2.5,5.8,1.8,Refurbish,RJ
4,359,7.6,3.0,6.6,2.1,Refurbish,RJ
5,9,4.9,3.1,1.5,0.1,Reject,RJ
6,277,7.2,3.6,6.1,2.5,Refurbish,SP


# Q1. Exploração e Preparação dos Dados

1. Explore seu data set.
2. Se houverem, elimine atributos que não devem fazer parte do modelo (treinamento).
3. Se existirem valores nulos elimine as linhas desses casos.
4. Faça o hot encode do atributo categórico `Unit` para o uso nos modelos. Em seguida exclua o atributo `Unit` original. 
5. NÃO NORMALIZE os dados ou faça outras transformações não solicitadas (troca da ordem de atributos por exemplo).





In [ ]:
df = df[ , -1 ]
head(df)

,A,B,C,D,Quality,Unit
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,4.9,3.1,1.5,0.1,Reject,SP
2,4.8,3.4,1.6,0.2,Reject,SP
3,6.7,2.5,5.8,1.8,Refurbish,RJ
4,7.6,3.0,6.6,2.1,Refurbish,RJ
5,4.9,3.1,1.5,0.1,Reject,RJ
6,7.2,3.6,6.1,2.5,Refurbish,SP


In [ ]:
sum(is.na(df))

[1] 19

In [ ]:
df = na.omit(df)

In [ ]:
arquivo = paste0('http://meusite.mackenzie.br/rogerio/TIC/pieces',f,'_new.csv')
X_new = read.csv(arquivo)
X_new


A,B,C,D,Unit
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
6.3,3.4,5.6,2.4,SP
6.7,2.5,5.8,1.8,RJ
5.2,4.1,1.5,0.1,BH
6.1,2.9,4.7,1.4,SP
4.8,3.4,1.9,0.2,BH


In [ ]:
df = dummy.data.frame('Unit',data=df,sep='_',drop=TRUE)
head(df)

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”


,A,B,C,D,Quality,Unit_BH,Unit_RJ,Unit_SP
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>
1,4.9,3.1,1.5,0.1,Reject,0,0,1
2,4.8,3.4,1.6,0.2,Reject,0,0,1
3,6.7,2.5,5.8,1.8,Refurbish,0,1,0
4,7.6,3.0,6.6,2.1,Refurbish,0,1,0
5,4.9,3.1,1.5,0.1,Reject,0,1,0
6,7.2,3.6,6.1,2.5,Refurbish,0,0,1


In [ ]:
cat('Nr de linhas e colunas após as transformações: ', nrow(df), ncol(df))

Nr de linhas e colunas após as transformações:  481 8

In [ ]:
cat('Frequencias do atributo Quality:\n')
table(df$Quality)

Frequencias do atributo Quality:



   Accept Refurbish    Reject 
      149       176       156 

# Q2-Q3. Crie e Avalie os diferentes Modelos de Classificação

Crie dois modelos de Classificação, um **Knn, com K=3** e uma **Árvore de Decisão**. Empregue o modelo de maior acurácia para predição dos novos casos. Em todos casos:

1. Empregue o `set.seed(1984)` para a geração dos dados de treinamento e teste.
2. Crie os conjuntos de treinamento e teste com 1/3 para teste.
3. Não empregue outros parâmetros não solicitados nos modelos.
4. Empregue em ambos os modelos os mesmos conjuntos de treinamento/teste (não recrie)




## Criação dos Conjuntos de Treinamento e Teste

In [ ]:
set.seed(1984)

L = sample( 1: nrow(df), round(nrow(df)/3) )
X_test = df[ L , -c(5) ]
y_test = df[ L , c('Quality') ]
X_train = df[ -L , -c(5) ]
y_train = df[ -L , c('Quality') ]

print(head(X_train))
print(head(y_train))

    A   B   C   D Unit_BH Unit_RJ Unit_SP
1 4.9 3.1 1.5 0.1       0       0       1
2 4.8 3.4 1.6 0.2       0       0       1
3 6.7 2.5 5.8 1.8       0       1       0
4 7.6 3.0 6.6 2.1       0       1       0
6 7.2 3.6 6.1 2.5       0       0       1
7 6.6 2.9 4.6 1.3       0       0       1
[1] "Reject"    "Reject"    "Refurbish" "Refurbish" "Refurbish" "Accept"   


## Modelo Knn, k=3

In [ ]:
y_pred = knn(X_train, X_test, y_train, k = 3)
cm = table(y_test, y_pred)
print(cm)
cat( "Acuracidade do modelo knn (k=3): ", sum(diag(cm))/sum(cm))

           y_pred
y_test      Accept Refurbish Reject
  Accept        43         7      0
  Refurbish      1        58      0
  Reject         0         0     51
Acuracidade do modelo knn (k=3):  0.95

## Modelo Árvore de Decisão

In [ ]:
Train = cbind(X_train,y_train)
names(Train)[8] = 'Quality'

fit = rpart(Quality ~ ., data=Train)
y_pred = predict(fit,X_test,type = "class")
cm = table(y_test, y_pred)
print(cm)
cat( "Acuracidade do modelo Decision Tree: ", sum(diag(cm))/sum(cm))

           y_pred
y_test      Accept Refurbish Reject
  Accept        43         7      0
  Refurbish      0        59      0
  Reject         0         0     51
Acuracidade do modelo Decision Tree:  0.95625

# Q4. Predição com o Melhor Modelo (Acurácia Geral)

## Preparação dos Dados

Dica: Hot encode dos dados? Cuidado para se não houver todos os valores nos novos casos.

In [ ]:
X_new = dummy.data.frame('Unit',data=X_new,sep='_',drop=TRUE)
head(X_new)

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”


,A,B,C,D,Unit_BH,Unit_RJ,Unit_SP
,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
1,6.3,3.4,5.6,2.4,0,0,1
2,6.7,2.5,5.8,1.8,0,1,0
3,5.2,4.1,1.5,0.1,1,0,0
4,6.1,2.9,4.7,1.4,0,0,1
5,4.8,3.4,1.9,0.2,1,0,0


In [ ]:
for (c in c('Unit_BH','Unit_RJ','Unit_SP')){
  if (match(c,names(X_new)) == 0) {
    X_new[c] = 0
  }
}  


## Predição

In [ ]:
if (f == '1'){
  print('Árvore de Decisão')
  y_pred = predict(fit,X_new,type = "class")
}

if (f == '2'){
  print('knn')
  y_pred = knn(X_train, X_new, y_train, k = 3)
}

print(y_pred)

[1] "Árvore de Decisão"
        1         2         3         4         5 
Refurbish Refurbish    Reject    Accept    Reject 
Levels: Accept Refurbish Reject


# Q5. Regressão Linear

Crie um modelo de Regressão Linear para o Cálculo da medida C das peças com base nas demais medidas. Empregue o modelo para estimar a medida dos C das novas peças.

In [ ]:
fit = lm(C ~ A + B + D, data=df)
summary(fit)




Call:
lm(formula = C ~ A + B + D, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.02066 -0.22294 -0.02886  0.22819  1.08015 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.34291    0.16714  -2.052   0.0407 *  
A            0.70183    0.03220  21.798   <2e-16 ***
B           -0.56808    0.03712 -15.305   <2e-16 ***
D            1.47043    0.03607  40.763   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3146 on 477 degrees of freedom
Multiple R-squared:  0.9683,	Adjusted R-squared:  0.9681 
F-statistic:  4858 on 3 and 477 DF,  p-value: < 2.2e-16


In [ ]:
y_pred = predict(fit,X_new)
print(y_pred)

       1        2        3        4        5 
5.676182 5.585929 1.124526 4.349427 1.388492 


In [ ]:
# Em Python... 

# import statsmodels.formula.api as sm

# model = sm.ols(formula='C ~ A + B + D', data=df)
# result = model.fit()

# y_pred = result.predict(X_new[['A','B','D']])

# print(result.summary())
# print(y_pred)
